# Converts recorded rosbags to hdf5 file

The data is recorded with the `rosbag` command. To train a neural network, it is more convienient to have
the data in the hdf5 file format. This notebook converts the data.

## Train and Test Split

Each rosbag is assigned either to the train or the test data. This is defined in the
`train.txt` and `test.txt` files. 

## Important Parameters

* `data_dir`: Writes data to this directory
* `rosbag_fname`: Filename of a rosbag file of which a video with steering and speed is created.
* `mode`: Must be either `train` or `test`. 

## Important: You have to run this notebook twice!

First with `mode = "train"` and then with `mode = "test"` to generate
both train and test hdf5 files. You may skip the video generation for the second run.


In [ ]:
%matplotlib inline
import rosbag
import rospy
import numpy as np
from sensor_msgs.msg import Image
import matplotlib.pyplot as plt
import imageio
from moviepy.editor import ImageSequenceClip
from scipy.misc import imsave, imread
import os
from StringIO import StringIO
import tempfile
from tqdm import tqdm
import shutil
import PIL
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.font_manager

from PIL import ImageDraw, ImageFont, ImageFilter
import matplotlib as mpl

import h5py 


In [ ]:
# test dataset
data_dir = '../data'
video_dir = os.path.join(data_dir, "video")
rosbag_dir = os.path.join(data_dir, 'rosbag')

# either train or test
mode = 'test'

rosbag_fname = os.path.join(rosbag_dir, '2017-05-31-14-20-54.bag')
rosbag_name, _ = os.path.splitext(os.path.basename(rosbag_fname))

def get_filenames(txt_file):
    return [l.rstrip('\n') for l in f.readlines()]
    
    
with open(os.path.join(rosbag_dir, "train.txt"), 'r') as f: 
    train_rosbags = get_filenames(f)
    
with open(os.path.join(rosbag_dir, "test.txt"), 'r') as f: 
    test_rosbags = get_filenames(f)
    
print("TRAIN", train_rosbags)
print("TEST", test_rosbags)

if mode == 'train':
    rosbag_fnames_to_convert = train_rosbags
elif mode == 'test':
    rosbag_fnames_to_convert = test_rosbags
else:
    raise Exception()
    
rosbag_fnames_to_convert = [os.path.join(rosbag_dir, f) for f in rosbag_fnames_to_convert]

rescale_factor = 8
tmp = os.path.abspath('../tmp/')
if not os.path.exists(tmp):
    os.makedirs(tmp)
    
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

    
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

In [ ]:
class topics:
    image_raw = "/app/camera/rgb/image_raw/compressed"
    image_resize = "/deepcar/resize_img80x60/compressed"
    lights = "/manual_control/lights"
    speed = "/manual_control/speed"
    steering = "/manual_control/steering"
    stop_start = "/manual_control/stop_start"
    yaw = "/model_car/yaw"
    twist = "/motor_control/twist" 
    odom = "/odom"
    
    labels = [speed, steering, yaw]
    

In [ ]:
def ros_to_numpy(msg):
    return imread(StringIO(msg.data))

def ros_to_pil(msg):
    return PIL.Image.open(StringIO(msg.data))

In [ ]:
bag = rosbag.Bag(rosbag_fname)
bag.get_start_time()

In [ ]:
def get_time(t):
    return (1-t)*bag.get_start_time() + t*bag.get_end_time()

print(bag.get_start_time())
print(get_time(0.2))

In [ ]:
tti = bag.get_type_and_topic_info()

fmt = "{:40}| {:30}| {:<20}"
print(fmt.format("topic", "message type", "count"))
print("-" * 80)
for name, topic in sorted(tti.topics.items()):
    print fmt.format(name, topic.msg_type, topic.message_count)

In [ ]:
def get_image_topic(bag):
    tti = bag.get_type_and_topic_info()

    if topics.image_raw in tti.topics.keys():
        return topics.image_raw
    
    assert topics.image_resize in  tti.topics.keys()
    return topics.image_resize 

image_topic = get_image_topic(bag)
image_topic

### Get all messages for lables and convert them to pandas dataframes

In [ ]:
def parse_msg(msg, msg_type):
    if msg_type in ("std_msgs/Int16", "std_msgs/Float32"):
        return msg.data
    elif msg_type == 'geometry_msgs/Twist':
        return {
            msg.linear.x,
        }
    else:
        raise Exception('unknown')
    
def get_labels_as_pandas(bag, topics=topics.labels):
    topic_data = {t: [] for t in topics}
    times = {t: [] for t in topics}
    for topic, msg, time in bag.read_messages(topics=topics):
        msg_type = tti.topics[topic].msg_type
        data = parse_msg(msg, msg_type)
        times[topic].append(time.to_sec())
        topic_data[topic].append(data)
    df = {}
    for k, data in topic_data.items():
        df[k] = pd.Series(data, index=pd.DatetimeIndex(pd.to_datetime(times[k], unit='s')))
    return df

In [ ]:
all_bags = train_rosbags + test_rosbags
fig, axes = plt.subplots(ncols=len(all_bags), figsize=(25, 7))
for ax, bag_fname in zip(axes, sorted(all_bags)):
    bag = rosbag.Bag(os.path.join(rosbag_dir, bag_fname))
    df = get_labels_as_pandas(bag, ['/manual_control/steering'])
    steering = df['/manual_control/steering']
    steering.hist(ax=ax)
    ax.set_title(bag_fname)
    
plt.show(fig)
plt.close(fig)

In [ ]:
bag = rosbag.Bag(rosbag_fname)
print(rosbag_fname)
df = get_labels_as_pandas(bag)

In [ ]:
frame_rate = tti.topics[image_topic].frequency
print("Frame rate is: {:.2f}".format(frame_rate))

In [ ]:
def command_at(df, time):
    if type(time) == rospy.rostime.Time:
        time = time.to_sec()
    dt = pd.to_datetime(time, unit='s')
    before = df[:dt]
    if len(before) != 0:
        return before[-1]
    else:
        return df[dt:][0]

## Create a video with steering, speed, and time [optional]

Visualizes the steering, speed and time in a video. This is completly optional and may be skipped.

In [ ]:
def image_draw_info(img, speed, steering, time, font=None):
    if font is None:
        fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
        mono_fonts = [f for f in fonts if "mono" in f.lower() and 'bold' in f.lower()]
        mono_font = mono_fonts[0]
        font = ImageFont.truetype(mono_font, 18)
    
    draw = ImageDraw.Draw(img)

    draw.text((10, 450), time, fill="#ffffff", font=font)
    draw.text((10, 20), "steering: {}".format(steering), fill="#00ff00", font=font)
    angle = (steering) / 180. * np.pi
    x = img.size[0] / 2
    y = 100
    draw.line([x, y, x +  60*np.cos(angle), y  - 60*np.sin(angle)], fill="#00ff00", width=3)
    draw.text((10, 50), "speed: {}".format(speed), fill='#ff0000', font=font)
    del draw

    
img_fnames = []
tmp_dir = tempfile.mkdtemp(dir=tmp)

for i, (topic, msg, time) in enumerate(tqdm(
    bag.read_messages(topics=image_topic, end_time=rospy.Time(get_time(0.5))))):
    
    img_fname = os.path.join(tmp_dir, "{:06d}.png".format(i))
    img = ros_to_numpy(msg)
    
    img = PIL.Image.fromarray(img)
    for name, d in df.items():
        value = command_at(d, time)
        
        if name == topics.steering:
            steering = value
        elif name == topics.speed:
            speed = value
    dtime = datetime.fromtimestamp(time.to_sec())
    image_draw_info(img, speed, steering, dtime.isoformat())
    img.save(img_fname)
    del img

In [ ]:
video = ImageSequenceClip(tmp_dir, fps=frame_rate, with_mask=False)
output_fname = os.path.abspath(os.path.join(video_dir, rosbag_name + "_controls.webm"))
video.write_videofile(output_fname, ffmpeg_params=['-b:v', '0', '-crf', '20'])
shutil.rmtree(tmp_dir)

## Create hdf5 file and convert all rosbag files

In [ ]:
def rescale_img(img):
    blur = ImageFilter.GaussianBlur(radius=rescale_factor * 1./3)
    return img.convert('L').filter(blur).resize(img_shape_pil, resample=PIL.Image.BILINEAR)

# numpy it is h, w and in the PIL world w, h
img_shape_pil = (80, 60)
img_shape = (60, 80)

In [ ]:
h5_fname = os.path.join(data_dir, mode + '.hdf5')
if os.path.exists(h5_fname):
    os.remove(h5_fname)
h5 = h5py.File(h5_fname)
print("saving h5 file to: " + h5_fname)

In [ ]:
chunk = 32
h5.create_dataset('image', shape=(1, ) + img_shape, dtype='uint8',
                  maxshape=(None,) + img_shape,
                  chunks=(64,) + img_shape)

h5.create_dataset('steering', shape=(1, 1), dtype='float32', maxshape=(None, 1), chunks=(chunk, 1))
h5.create_dataset('timestamp', shape=(1, 1), dtype='float64', maxshape=(None, 1), chunks=(chunk, 1))

In [ ]:
steering_norm = np.array([1 / 180. * np.pi, -np.pi/2])
h5['steering'].attrs['normalize'] = steering_norm

In [ ]:
def normalize(normalize, value):
    return np.dot(normalize, [value, 1])

i = 0
print("Filling h5 file {} with data from {}".format(h5_fname, rosbag_fnames_to_convert))
for fname in rosbag_fnames_to_convert:
    bag = rosbag.Bag(fname)
    image_topic = get_image_topic(bag)
    dfs = get_labels_as_pandas(bag, ['/manual_control/steering'])
    steering_df = dfs['/manual_control/steering']
    for topic, msg, time in tqdm(bag.read_messages(topics=image_topic)):
        for dset in h5.values():
            dset.resize(size=i+1, axis=0)
        
        img = ros_to_pil(msg)
        if image_topic == topics.image_raw:
            h5['image'][i] = np.array(rescale_img(img))
            h5['steering'][i] = normalize(steering_norm, command_at(steering_df, time))
        elif image_topic == topics.image_resize:
            h5['image'][i] = np.array(img.convert('L'))
            steering = command_at(steering_df, time)
            h5['steering'][i] = normalize(steering_norm, 180 - steering)
        else:
            raise Exception()
        h5['timestamp'][i] = time.to_sec()
        i += 1

In [ ]:
print("{:20}| {:10}| {:30}".format("name", "dtype", "shape"))
print("-" * 40)
for name, dset in h5.items():
    print("{:20}| {:10}| {:30}".format(name, dset.dtype, dset.shape))

In [ ]:
step = 100
images = [h5['image'][i*step] for i in range(10)]
PIL.Image.fromarray(np.concatenate(images, axis=1))

In [ ]:
h5.close()